In [125]:
import numpy as np
import pandas as pd

In [246]:
from library_w19_deep_2 import *

%who function

ann_flex	 ann_flex_predictor	 ann_flex_tester	 ann_predictor	 ann_simple	 ann_simple_batch	 ann_tester	 from_scratch	 from_scratch_batch	 
from_scratch_flex	 mse	 mse_der	 sigmoid	 sigmoid_der	 


In [259]:
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Project/HistoricalQuotes.csv', 'r') as f:
  quote_table = pd.read_csv(f)

In [260]:
quote_table = quote_table.drop([0])
quote_table.head(1)

,date,close,volume,open,high,low
1,2019/03/18,23.25,34636110.0000,23.3,23.62,23.04


In [262]:
bins = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35']
#bins2 = [0, 5, 10, 15, 20, 25, 30, 35]
#bins3 = pd.IntervalIndex.from_tuples([(0,3), (3,6), (6,9), (9,12), (12,15), (15,18), (18,21), (21,24), (24,27), (27,30), (30,33)])
tuples = []
"""
for i in range(33):
    #tup = (i, i+1)
    tuples.append((i, i+1))
bins3 = pd.IntervalIndex.from_tuples(tuples)
"""


quote_table['close_binned'] = pd.cut(quote_table['close'], bins)

#11 output nodes, corresponding to the one-hot encoding of close_binned
one_hot_close_binned = pd.get_dummies(quote_table["close_binned"], prefix = "close", dummy_na = False)
quote_table = quote_table.join(one_hot_close_binned)
quote_table = quote_table.dropna()
labels = quote_table[['close_(0, 3]','close_(3, 6]','close_(6, 9]','close_(9, 12]','close_(12, 15]','close_(15, 18]','close_(18, 21]','close_(21, 24]','close_(24, 27]','close_(27, 30]','close_(30, 33]']].values
"""
amount = 33
labels_list = []
for i in range(amount):
    lab = "close_(" 
    lab += str(i) 
    lab += ", " 
    lab +=str(i+1) 
    lab += ']'
    labels_list.append(lab)  
print(labels_list)
labels = quote_table[labels_list].values
"""

#normalize columns
for i in range(len(quote_table["volume"])):
    quote_table['volume'][i+1] = int(quote_table["volume"][i+1][:-5])
quote_table["volume_normed"] = quote_table["volume"] / quote_table["volume"].max()
quote_table["open_normed"] = quote_table["open"] / quote_table["open"].max()
quote_table["high_normed"] = quote_table["high"] / quote_table["high"].max()
quote_table["low_normed"] = quote_table["low"] / quote_table["low"].max()

ann_table = quote_table[['volume_normed', 'open_normed', 'high_normed', 'low_normed']]
#ann_table = quote_table.dropna()
feature_set = ann_table.values

print(feature_set.shape)
feature_set[:5]



/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(2517, 4)


array([[0.10695290486637878, 0.7022302591922845, 0.6918570591681312,
        0.7157502329916123],
       [0.1434179366949261, 0.6962025316455697, 0.6927357937902753,
        0.7148182665424045],
       [0.13187603927193453, 0.7043399638336347, 0.68804920913884,
        0.7082945013979497],
       [0.1741556518637803, 0.7130801687763714, 0.7073813708260105,
        0.72538055296676],
       [0.17372945966351133, 0.6962025316455697, 0.6969830111306385,
        0.7076731904318112]], dtype=object)

In [263]:
quote_table.head(1)

,date,close,volume,open,high,low,close_binned,"close_(0, 1]","close_(1, 2]","close_(2, 3]",...,"close_(15, 18]","close_(18, 21]","close_(21, 24]","close_(24, 27]","close_(27, 30]","close_(30, 33]",volume_normed,open_normed,high_normed,low_normed
1,2019/03/18,23.25,34636110,23.3,23.62,23.04,"(21, 24]",0,0,0,...,0,0,1,0,0,0,0.106953,0.70223,0.691857,0.71575


In [264]:
quote_table.describe()

,close,open,high,low,"close_(0, 1]","close_(1, 2]","close_(2, 3]","close_(3, 4]","close_(4, 5]","close_(5, 6]",...,"close_(12, 15]","close_(15, 18]","close_(18, 21]","close_(21, 24]","close_(24, 27]","close_(27, 30]","close_(30, 33]",open_normed,high_normed,low_normed
count,2517.000000,2517.000000,2517.000000,2517.000000,2517.0,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,...,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000,2517.000000
mean,7.285864,7.287811,7.446598,7.122315,0.0,0.037346,0.163290,0.152563,0.099325,0.061184,...,0.081843,0.016687,0.024632,0.017084,0.007549,0.004768,0.005959,0.219645,0.218119,0.221259
std,5.426202,5.426835,5.567684,5.275762,0.0,0.189646,0.369703,0.359637,0.299157,0.239715,...,0.274181,0.128120,0.155033,0.129610,0.086572,0.068897,0.076983,0.163557,0.163084,0.163894
min,1.620000,1.620000,1.690000,1.610000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048825,0.049502,0.050016
25%,3.550000,3.550000,3.630000,3.470000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.106992,0.106327,0.107797
50%,5.790000,5.780000,5.930000,5.680000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.174201,0.173697,0.176452
75%,9.110000,9.080000,9.300000,8.940000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.273659,0.272408,0.277726
max,32.720000,33.180000,34.140000,32.190000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [270]:
(weights, biases) = from_scratch_flex(feature_set, labels, hypers={'epochs':3000, 'cost-reporting': 1000})
#print(weights)
#print(biases)

(0, 0.3398078189173233)
(1000, 0.3688432180810744)


KeyboardInterrupt: 

In [ ]:
predictions = ann_flex_tester(feature_set, weights, biases)
prediction_list = []
accuracy = 0

for i in range(len(predictions)):
    
    mx = predictions[i].tolist().index(max(predictions[i].tolist()))
    #print(mx)
    pred = []
    for k in range(11):
        pred.append(0)
    pred[mx] = 1
    
    prediction_list.append(pred)

for j in range(len(prediction_list)):
    print(prediction_list[i])
    print(list(labels[i]))
    print("*"*30)
    if(prediction_list[j] == list(labels[j])):
        accuracy += 1
        
accuracy = accuracy / len(prediction_list)
print(accuracy)
    
    